# Ownership

## Stack vs Heap
- **Stack**: The stack stores values in the order it gets them and removes the values in the opposite order. This is referred to as last in, first out. *All data stored on the stack must have a known, fixed size*. *Data with an unknown size at compile time or a size that might change must be stored on the heap instead*.  
- **Heap**: The heap is less organized: when you put data on the heap, you request a certain amount of space. The memory allocator finds an empty spot in the heap that is big enough, marks it as being in use, and returns a pointer, which is the address of that location (the whole process is called *allocating*). Because the pointer to the heap is a known, fixed size, you can *store the pointer on the stack*, but when you want the actual data, you must follow the pointer.
- **Performance Comparison**:
    - Pushing to the stack is faster than allocating on the heap because the allocator never has to search for a place to store new data
    - Accessing data in the heap is slower than accessing data on the stack because you have to follow a pointer to get there

When your code calls a function, the values passed into the function (including, potentially, pointers to data on the heap) and the function’s local variables get pushed onto the stack. When the function is over, those values get popped off the stack.


## What does the Rust's Ownership do?
- Keeps track of what parts of code are using what data on the heap
- Minimizing the amount of duplicate data on the heap
- Cleaning up unused data on the heap so you don’t run out of space

## Ownership Rules
- Each value in Rust has an owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

## Variable Scope
```rust
fn main() {
    {                      // s is not valid here, it’s not yet declared
        let s = "hello";   // s is valid from this point forward

        // do stuff with s
    }                      // this scope is now over, and s is no longer valid
}
```

### Example: `String` type
The types covered previously are of a known size, can be stored on the stack and popped off the stack when their scope is over, and can be quickly and trivially copied to make a new, independent instance if another part of code needs to use the same value in a different scope. But we want to look at data that is stored on the heap and explore how Rust knows when to clean up that data, and the `String` type is a great example.

In [7]:
fn main() {
    let mut s = String::from("hello");
    s.push_str(", world!"); // push_str() appends a literal to a String
    println!("{}", s); // This will print `hello, world!`
}
main()

hello, world!


()

With the `String` type, in order to support a mutable, growable piece of text, we need to allocate an amount of memory on the heap, unknown at compile time, to hold the contents. This means:
- The memory must be requested from the memory allocator at runtime.
- We need a way of returning this memory to the allocator when we’re done with our `String`. 


### Memory and Allocation
Since Rust does not have a garbage collector, and making programmers freeing the `String` manually (e.g. C) has produced many bugs, Rust takes a different path: <span style="color:green">**the memory is automatically returned once the variable that owns it goes out of scope (it's dropped)</span>.**

```rust
fn main() {
    {
        let s = String::from("hello"); // s is valid from this point forward

        // do stuff with s
    }                                  // this scope is now over, and s is no longer valid
                                       // Rust calls drop automatically at the closing curly bracket
}
```

## Variables and Data Interacting with Move

Below we bind the value 5 to `x`; then make a copy of the value in `x` and bind it to `y`

In [17]:
let x = 5;
let y = x;
println!("{:p}, {}", &x, x);
println!("{:p}, {}", &y, y);

0x7ffd57c8ec18, 5
0x7ffd57c8ec1c, 5


<img src="./images/1.png" style="width: 30%;">

We have a similar syntax below, but under the hood things are totally different

In [15]:
let s1 = String::from("hello");
let s2 = s1;
println!("{:p}, {}", &s1, s1);
println!("{:p}, {}", &s2, s2);

Error: borrow of moved value: `s1`

When we assign `s1` to `s2`, we copy the pointer, the length, and the capacity that are on the stack, but we do not copy the data on the heap that the pointer refers to
<img src="./images/2.svg" style="width: 80%;">

When `s2` and `s1` go out of scope, they will both try to free the same memory. This is known as a double free error and is one of the memory safety bugs we mentioned previously. **Freeing memory twice can lead to memory corruption, which can potentially lead to security vulnerabilities.**

<span style="color:green">**To ensure memory safety, after the line `let s2 = s1;`, Rust considers `s1` as no longer valid (move the data pointed to by `s1` to `s2`, or we transfer the data ownership of `s1` to `s2`). Therefore, Rust doesn’t need to free anything when `s1` goes out of scope**</span>
<img src="./images/3.svg" style="width: 80%;">

Hence, this solves our problem! With only `s2` valid, when it goes out of scope it alone will free the memory, and we’re done.

## Variables and Data Interacting with `clone`

If we do want to **deeply copy the heap data of the `String`**, not just the stack data, we can use a common method called `clone`.

In [18]:
let s1 = String::from("hello");
let s2 = s1.clone();

println!("{:p}, {}", &s1, s1);
println!("{:p}, {}", &s2, s2);

0x7ffd57c8ec28, hello


0x7ffd57c8ec60, hello


### Stack-Only Data: Copy
Types such as integers that have a known size at compile time are stored entirely on the stack, so copies of the actual values are quick to make. Hence, there’s no difference between deep and shallow copying here, so calling `clone` wouldn’t do anything different from the usual shallow copying, and we can leave it out.

In [21]:
let x = 5;
let y = x;
let z = x.clone();

println!("{:p}, {}", &x, x);
println!("{:p}, {}", &y, y);
println!("{:p}, {}", &z, z);

0x7ffd57c8ebec, 5
0x7ffd57c8ebf0, 5
0x7ffd57c8ebf4, 5


#### The `Copy` Trait
Rust has a special annotation called the `Copy` trait that we can place on types that are stored on the stack, as integers are. If a type implements the `Copy` trait, variables that use it do not move, but rather are trivially copied, making them still valid after assignment to another variable.  
Rust won’t let us annotate a type with `Copy` if the type, or any of its parts, has implemented the `Drop`.  
As a general rule, any group of simple scalar values can implement `Copy`, and nothing that requires allocation or is some form of resource can implement `Copy`, for example:
- All the integer types, such as `u32`.
- The Boolean type, `bool`, with values `true` and `false`.
- All the floating-point types, such as `f64`.
- The character type, `char`.
- `Tuples`, if they only contain types that also implement `Copy`. For example, `(i32, i32)` implements `Copy`, but `(i32, String)` does not.

## Ownership and Function

### Passing values into a function and scope
The mechanics of passing a value to a function are similar to those when assigning a value to a variable.   
<span style="color:green">**Passing a variable to a function will move or copy, just as assignment does.**</span>

In [32]:
fn main() {
    let s = String::from("hello");  // s comes into scope
    takes_ownership(s);             // s's value moves into the function...
                                    // ... and so is no longer valid here
    println!("{}", s);              // this will return error
    let x = 5;                      // x comes into scope
    makes_copy(x);                  // x would move into the function,
                                    // but i32 is Copy, so it's okay to still
                                    // use x afterward
    println!("{}", x);              // this is ok

} // Here, x goes out of scope, then s. But because s's value was moved, nothing
  // special happens.

fn takes_ownership(some_string: String) { // some_string comes into scope
    println!("{}", some_string);
} // Here, some_string goes out of scope and `drop` is called. The backing
  // memory is freed.

fn makes_copy(some_integer: i32) { // some_integer comes into scope
    println!("{}", some_integer);
} // Here, some_integer goes out of scope. Nothing special happens.

main()

Error: borrow of moved value: `s`

### Return Values and Scope
Returning values can also transfer ownership.

In [46]:
fn main() {
    let s1 = gives_ownership();         // gives_ownership moves its return
                                        // value into s1
    let s2 = String::from("hello");     // s2 comes into scope
    let s3 = takes_and_gives_back(s2);  // s2 is moved into
                                        // takes_and_gives_back, which also
                                        // moves its return value into s3
                                        // takes_and_gives_back does the same thing with let s3 = s2
    println!("{}", s3);                 // we can print out s3
    // println!("{}", s2);                 // we can't print out s2
} // Here, s3 goes out of scope and is dropped. s2 was moved, so nothing
  // happens. s1 goes out of scope and is dropped.

fn gives_ownership() -> String {             // gives_ownership will move its
                                             // return value into the function
                                             // that calls it
    let some_string = String::from("yours"); // some_string comes into scope
    some_string                              // some_string is returned and
                                             // moves out to the calling
                                             // function
}

// This function takes a String and returns one
fn takes_and_gives_back(a_string: String) -> String { // a_string comes into scope
    a_string  // a_string is returned and moves out to the calling function
}

main()

hello


()

Rust does let us return multiple values using a tuple,

In [34]:
fn main() {
    let s1 = String::from("hello");

    let (s2, len) = calculate_length(s1);

    println!("The length of '{}' is {}.", s2, len);

}

fn calculate_length(s: String) -> (String, usize) {
    let length = s.len(); // len() returns the length of a String

    (s, length)
}
main()

The length of 'hello' is 5.


()

<span style="color:green">**The ownership of a variable follows the same pattern every time: assigning a value to another variable moves it. When a variable that includes data on the heap goes out of scope, the value will be cleaned up by `drop` unless ownership of the data has been moved to another variable.**</span>

# Reference & Borrowing
Taking ownership and then returning ownership with every function is a bit tedious. What if we want to let a function use a value but not take ownership? 
Luckily for us, Rust has a feature for <span style="color:red">**using a value without transferring ownership, called references, which gives you a way to let multiple parts of your code access one piece of data without needing to copy that data into memory multiple times.**</span>

## Introduction to References and Borrowing

Consider the below code: What if after calculating the length of `s1`, we want to use it?

In [40]:
fn main() {
    let s1 = String::from("hello");
    let len = calculate_length(s1);
    println!("The length of '{}' is {}.", s1, len);  // this does not work
}

fn calculate_length(s: String) -> usize {
    println!("Calculating the length of '{}': ", s);  // this works
    s.len()
}

main()

Error: borrow of moved value: `s1`

Since `s1`'s ownership is moved into `calculate_length`, we got the error. To fix this, *we can provide a reference to the `String` value (using the `&` literal).* 

<span style="color:green">**A reference is like a pointer in that it’s an address we can follow to access the data stored at that address; that data is owned by some other variable. Unlike a pointer, a reference is guaranteed to point to a valid value of a particular type for the life of that reference.**<span style="color:green">

In [43]:
fn main() {
    let s1 = String::from("hello");
    let len = calculate_length(&s1);  // The &s1 syntax lets us create a reference 
                                      // that refers to the value of s1 but does not own it
    println!("The length of '{}' is {}.", s1, len);
}

fn calculate_length(s: &String) -> usize {  // s is a reference to a String
    println!("Calculating the length of '{}': ", s);  // this works
    s.len()
} // Here, s goes out of scope. But because it does not have ownership of what
  // it points / refers to, it is not dropped.

main()

Calculating the length of 'hello': 
The length of 'hello' is 5.


()

`&String` is a reference to a `String`. These ampersands represent references, and they allow you to refer to some value without taking ownership of it  

<img src="./images/4.svg" style="width: 50%;">


*Note: The opposite of referencing by using `&` is dereferencing, which is accomplished with the dereference operator, `*`*

<span style="color:green">*We call the action of creating a reference **borrowing**. As in real life, if a person owns something, you can borrow it from them. When you’re done, you have to give it back. You don’t take ownership of it.*</span>  

So:
$\text{creating reference} = \text{borrowing} = \text{not taking ownership}$

**If a variable is immutable, so is it reference. We can't modify an immutable variable we borrow**:

In [47]:
fn main() {
    let s = String::from("hello");

    change(&s);
}

fn change(some_string: &String) {
    some_string.push_str(", world");
}

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

However, if a variable is `mut`able, we can modify it through a mutable reference (`&mut`):

In [50]:
fn main() {
    let mut s = String::from("hello");
    println!("{}", s);
    change(&mut s);
    println!("{}", s);
}

fn change(some_string: &mut String) {
    println!("modifying the string...");
    some_string.push_str(", world");
}

main()

hello
modifying the string...
hello, world


()

<span style="color:green">**Mutable Reference Rule: If you have a mutable reference to a value, you can have no other references to that value (including immutable ones)**</span> Why? To prevent data races at compile time. A data race is similar to a race condition and happens when these three behaviors occur:
- Two or more pointers access the same data at the same time.
- At least one of the pointers is being used to write to the data.
- There’s no mechanism being used to synchronize access to the data.

In [65]:
fn main() {
    let mut s = String::from("hello");
    let r1 = &s; // no problem
    let r2 = &s; // no problem
    println!("{}, {}", r1, r2);
    let r3 = &mut s; // BIG PROBLEM
    println!("{}, {}, and {}", r1, r2, r3); // won't work
}

main()

Error: cannot borrow `s` as immutable because it is also borrowed as mutable

In [74]:
fn main() {
    let mut s = String::from("hello");
    let r1 = &mut s; // no problem
    let r2 = &s; // problem
    println!("{}, {}", r1, r2);
}
main()

Error: cannot borrow `s` as immutable because it is also borrowed as mutable

We can use curly brackets to create a new scope, allowing for multiple mutable references, just not simultaneous one:

In [58]:
fn main() {
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
        println!("{}", &r1);
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
}

main()

hello


()

*Multiple immutable references are allowed because no one who is just reading the data has the ability to affect anyone else’s reading of the data.*

**Note that a reference’s scope starts from where it is introduced and continues through the last time that reference is used**. For instance, the below code will compile because the scopes of the immutable references `r1` and `r2` end after the `println!` where they are last used, which is before the mutable reference `r3` is created

In [78]:
fn main() {
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    println!("{} and {}", r1, r2);
    // variables r1 and r2 will not be used after this point
    let r3 = &mut s; // no problem
    println!("{}", r3);  // this will work
    // println!("{} and {}", r1, r2);  // however, this will create errors

}

main()

hello and hello
hello


()

An example from Rustlings exercise

In [85]:
fn main() {
    let data = "Rust is great!".to_string();
    get_char(&data);
    println!("inside main: {}", data);  // this should work
    string_uppercase(data);
    // println!("inside main {}", data);  // this should not work
}

// Should not take ownership
fn get_char(data: &String) -> char {
    data.chars().last().unwrap()
}

// Should take ownership
fn string_uppercase(mut data: String) {
    data = data.to_uppercase();
    println!("inside string_uppercase: {}", data);
}

main()

inside main: Rust is great!
inside string_uppercase: RUST IS GREAT!


()

## Dangling References

In languages with pointers, it’s easy to erroneously create <span style="color:orange">*a dangling pointer—a pointer that references a location in memory that may have been given to someone else—by freeing some memory while preserving a pointer to that memory*</span>. 

<span style="color:lightgreen">*In Rust, by contrast, the compiler guarantees that references will never be dangling references: if you have a reference to some data, the compiler will ensure that the data will not go out of scope before the reference to the data does.*</span>

Below, we try to make a dangling pointer by making a function `dangle` that creats a `String` `s`, when the code of `dangle` is finished, `s` will be deallocated. But we tried to return a reference to it. That means this reference would be pointing to an invalid `String`. That’s no good! Rust won’t let us do this.

In [80]:
fn main() {
    let reference_to_nothing = dangle();
}

fn dangle() -> &String {  // returns a reference to a String
    let s = String::from("hello");  // s is a new String
    &s
}

Error: missing lifetime specifier

The solution is to return the `String` directly to move the ownership out:

In [81]:
fn main() {
    let string = no_dangle();
    println!("{}", string);
}

fn no_dangle() -> String {
    let s = String::from("hello");
    s
}

main()

hello


()

# The Slice Type

<span style="color:green">**Slices let you reference a contiguous sequence of elements in a collection rather than the whole collection. A slice is a kind of reference, so it does not have ownership**</span>

## `String` Slice
For example, a string slice is a reference to part of a `String`, and it looks like this:

In [92]:
fn main() {
    let s = String::from("hello world");

    let hello = &s[0..5];
    let world = &s[6..11];

    println!("{}", hello);
    println!("{}", world);
}

main()

hello
world


()

We create slices using a range within brackets by specifying `[starting_index..ending_index]`, where `starting_index` is the first position in the slice and `ending_index` is *one more than the last position* in the slice

<img src="./images/5.svg" style="width: 40%;">



In [95]:
#![allow(unused)]
fn main() {
    let s = String::from("hello");
    let len = s.len();
    
    let slice = &s[0..2];
    println!("{}", slice);
    let slice = &s[..2];
    println!("{}", slice);

    let slice = &s[3..len];
    println!("{}", slice);
    let slice = &s[3..];
    println!("{}", slice);

    let slice = &s[0..len];
    println!("{}", slice);
    let slice = &s[..];
    println!("{}", slice);
}
main()

he
he
lo
lo
hello
hello


()

## String Literal as Slices
Example: write a function that takes a string of words separated by spaces and returns the first word it finds in that string

In [2]:
fn first_word(s: &str) -> &str {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }

    &s[..]
}

fn main() {
    let my_string = String::from("hello world");
    // `first_word` works on slices of `String`s, whether partial or whole
    let word = first_word(&my_string[0..6]);
    println!("{}", word);
    let word = first_word(&my_string[..]);
    println!("{}", word);
    // `first_word` also works on references to `String`s, which are equivalent
    // to whole slices of `String`s
    let word = first_word(&my_string);
    println!("{}", word);

    let my_string_literal = "hello world";
    // `first_word` works on slices of string literals, whether partial or whole
    let word = first_word(&my_string_literal[0..6]);
    println!("{}", word);
    let word = first_word(&my_string_literal[..]);
    println!("{}", word);
    // Because string literals *are* string slices already,
    // this works too, without the slice syntax!
    let word = first_word(my_string_literal);
    println!("{}", word);
}

main()

hello
hello
hello
hello
hello
hello


()

## Other Slices

In [13]:
{
    let a = [1, 2, 3, 4, 5];
    let slice: &[i32] = &a[1..3];
    assert_eq!(slice, &[2, 3]);
}


()

`slice` has the type `&[i32]`. We will use this kind of slice for all sorts of other collections

# Summary

<span style="color:lightgreen">**The concepts of ownership, borrowing, and slices ensure memory safety in Rust programs at compile time. The Rust language gives you control over your memory usage in the same way as other systems programming languages, but having the owner of data automatically clean up that data when the owner goes out of scope means you don’t have to write and debug extra code to get this control**.</span>